# Ship Trajectory Prediction

In [ ]:
%matplotlib inline

In [ ]:
import urllib
import os
import numpy as np
import pandas as pd
import contextily as ctx
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from fiona.crs import from_epsg
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import sys
sys.path.append("..")
import movingpandas as mpd

import warnings
warnings.simplefilter("ignore")

In [ ]:
PAST = timedelta(minutes=2)
FUTURE = timedelta(minutes=2)
CRS_METRIC = from_epsg(31256)
CRS_LATLON = from_epsg(4326)

## Introduction

Trajectory prediction aims to predict the future position of a vessel based on its current position and past movement.
In contrast to other moving objects, such as land vehicles and aircraft, ships typically exhibit slow parabolic maneuvers. 
Commonly used physical model-based prediction methods include linear and kinematic prediction.

**Linear prediction** assumes that the object will continue in the previously observed direction with constant speed. Direction and speed are computed between the observed trajectory segments's start and end location.

**Kinematic prediction** considers the rate of change in speed (acceleration or deceleration) as well as direction. The method implemented for this work is based on Sang et al. (2016). 

In [ ]:
def plot_linear_and_kinetic_results(traj, futures):
    ax = traj.plot()
    ax = traj.df.plot(ax=ax)
    lin_predictor = mpd.TrajectoryPredictor(traj)
    kin_predictor = mpd.TrajectoryPredictor(traj)
    for future in futures:
        lin_result = lin_predictor.predict_linearly(future)
        ax = GeoDataFrame([lin_result]).rename(columns={0:'geometry'}).plot(ax=ax, color='orange')
        kin_result = kin_predictor.predict_kinetically(future)
        ax = GeoDataFrame([kin_result]).rename(columns={0:'geometry'}).plot(ax=ax, color='green')
    #print(kin_predictor.traj.df)

### Prediction example 1: accelerating straight trajectory 

This example demonstrates the prediction results for an example trajectory with an observed increase in speed but without any observed change in direction. 

In [ ]:
df = pd.DataFrame([
    {'geometry': Point(0.00, 0.00), 't': datetime(2018, 1, 1, 12, 0, 0)},
    {'geometry': Point(0.01, 0.01), 't': datetime(2018, 1, 1, 12, 1, 0)},
    {'geometry': Point(0.03, 0.03), 't': datetime(2018, 1, 1, 12, 2, 0)}
    ]).set_index('t')
geo_df = GeoDataFrame(df, crs=CRS_LATLON)
straight_traj = mpd.Trajectory(geo_df, 1)
straight_traj.add_speed()
plot_linear_and_kinetic_results(straight_traj, [timedelta(minutes=x) for x in range(1,6)])

### Prediction example 2: decelerating curved trajectory

This example demonstrates the prediction results for a curved example trajectory with an observed decrease in speed.

In [ ]:
df = pd.DataFrame([
    {'geometry': Point(0.00, 0.00), 't': datetime(2018, 1, 1, 12, 0, 0)},
    {'geometry': Point(0.01, 0.00), 't': datetime(2018, 1, 1, 12, 0, 30)},
    {'geometry': Point(0.02, 0.01), 't': datetime(2018, 1, 1, 12, 2, 0)},
    {'geometry': Point(0.02, 0.02), 't': datetime(2018, 1, 1, 12, 3, 0)}
    ]).set_index('t')
geo_df = GeoDataFrame(df, crs=CRS_LATLON)
curved_traj = mpd.Trajectory(geo_df, 1)
plot_linear_and_kinetic_results(curved_traj, [timedelta(minutes=x) for x in range(1,6)])

## Loading AIS data & creating trajectories

This work uses AIS data published by the Danish Maritime Authority. The AIS record sample extracted for this work covers vessel traffic on the 5th July 2017 near Gothenburg.

In [ ]:
df = read_file('data/demodata_ais.gpkg')
wgs84 = df.crs
print("Finished reading {}.".format(len(df)))

In [ ]:
df['t'] = pd.to_datetime(df['Timestamp'], format='%d/%m/%Y %H:%M:%S')
df = df.set_index('t')

In [ ]:
print("Original size: {} rows".format(len(df)))
df = df[df.SOG>0]
print("Reduced to {} rows after removing records with speed=0.".format(len(df)))

In [ ]:
MIN_LENGTH = 100 # meters
traj_collection = mpd.TrajectoryCollection(df, 'MMSI', MIN_LENGTH)
print("Finished creating {} trajectories.".format(len(traj_collection)))

In [ ]:
time_gap_to_split = timedelta(minutes=5)
trips = traj_collection.split_by_observation_gap(time_gap_to_split)
print("Extracted {} individual trips from {} continuous vessel tracks based on a time gap of {}.".format(len(trips), len(traj_collection), time_gap_to_split))

In [ ]:
#trips.plot(with_basemap=True, figsize=(12,9), linewidth=1, capstyle='round')

## Sampling

In [ ]:
sampler = mpd.TrajectoryCollectionSampler(traj_collection)

In [ ]:
samples = sampler.get_sample(PAST, FUTURE, randomize=True, fixed_seed=10)
print("Extracted {} samples.".format(len(samples)))

In [ ]:
trajs = [sample.past_traj for sample in samples]
sample_collection = mpd.TrajectoryCollection(trajs)
sample_collection.plot(with_basemap=True, figsize=(12,9), linewidth=1, capstyle='round')

## Linear prediction

In [ ]:
def get_predictions(trajs, linearly=True):
    predictions = []
    for traj in trajs:
        predictor = mpd.TrajectoryPredictor(traj)
        if linearly:
            result = predictor.predict_linearly(FUTURE)
        else:
            try:
                result = predictor.predict_kinetically(FUTURE)
            except:
                result = None
        predictions.append(result)
    return predictions

predictions = get_predictions(trajs)

predictions = GeoDataFrame(pd.DataFrame(predictions), crs=trajs[0].crs).rename(columns={0:'geometry'})

### Comparison of prediction and ground truth

In [ ]:
future_pos = [sample.future_pos for sample in samples]
future_pos = GeoDataFrame(pd.DataFrame(future_pos), crs=trajs[0].crs).rename(columns={0:'geometry'})
future_traj = [sample.future_traj for sample in samples]
future_traj = mpd.TrajectoryCollection(future_traj)

In [ ]:
ax = predictions.to_crs(epsg=3857).plot(figsize=(12,9), color='red')
ax = future_pos.to_crs(epsg=3857).plot(ax=ax, color='green')
ctx.add_basemap(ax, url=ctx.sources.ST_TERRAIN)
ax = future_traj.plot(ax=ax, for_basemap=True, linewidth=1, color='green')
sample_collection.plot(ax=ax, for_basemap=True, linewidth=3, capstyle='round')

## Kinetic prediction

## Evaluation of linear & kinetic prediction

In [ ]:
PAST = timedelta(seconds=60)
FUTURES = [timedelta(minutes=x) for x in range(1,6)]

In [ ]:
def evaluate(samples, predictions):
    errors = []
    for i, sample in enumerate(samples):
        if predictions[i]:
            evaluator = mpd.TrajectoryPredictionEvaluator(sample, predictions[i], CRS_METRIC, CRS_LATLON)
            errors.append(evaluator.get_distance_error())    
    return errors

def compute_distance_errors(future):
    samples = sampler.get_sample(PAST, future, randomize=True, fixed_seed=10)
    trajs = [sample.past_traj for sample in samples]
    
    linear_predictions = get_predictions(trajs)
    kinetic_predictions = get_predictions(trajs, linearly=False)
    
    lin_errors = evaluate(samples, linear_predictions)
    kin_errors = evaluate(samples, kinetic_predictions)
        
    return np.array(lin_errors), np.array(kin_errors)

In [ ]:
lin_errors = []
kin_errors = []

for future in FUTURES:
    e_lin, e_kin = compute_distance_errors(future)
    lin_errors.append(np.median(e_lin))
    print("Linear prediction for {} - median distance error {} (n={})".format(future, round(np.median(e_lin), 1), len(e_lin)))
    kin_errors.append(np.median(e_kin))
    print("Kinetic prediction for {} - median distance error {} (n={})".format(future, round(np.median(e_kin), 1), len(e_kin)))

In [ ]:
ax = plt.plot(lin_errors)
ax = plt.plot(kin_errors, color='red')

**References**
* Sang, L. Z., Yan, X. P., Wall, A., Wang, J., & Mao, Z. (2016). CPA calculation method based on AIS position prediction. The Journal of Navigation, 69(6), 1409-1426.